#### Looking up every item in a dict is killing performance on large queries.
#### Gotta find a better way, even if it tanks performance on remove() and/or update() ops.

In [46]:
import random
import time
import pandas as pd
import numpy as np
import sortednp as snp

In [3]:
class Thing:
    def __init__(self):
        self.x = random.random()

things = [Thing() for _ in range(10**7)]
obj_ids = [id(t) for t in things]
obj_dict = {id(t): t for t in things}

In [5]:
# make a subset containing most of the objects
some_idxs = list(set(random.choice(range(len(things))) for _ in range(5*10**7)))
some_ids = [obj_ids[k] for k in some_idxs]
print('subset size: ', len(some_ids) / len(obj_ids))

subset size:  0.9932565


In [6]:
# test dict and list lookup time for subset. 
# This is the best we can do with raw Python.
t0 = time.time()
s1 = [obj_dict[obj_id] for obj_id in some_ids]
t1 = time.time()
s2 = [things[idx] for idx in some_idxs]
t2 = time.time()

print('list lookup:', t2-t1)
print('dict lookup:', t1-t0)

list lookup: 1.419527530670166
dict lookup: 2.361854314804077


In [ ]:
# The SortedCollections are a lot slower than either. Tried SortedList and SortedSet.
# Too many lookups. Omitted.

In [7]:
# what about making a dataframe to do the lookups?
# nope -- worse than dict (probably similar algorithm though)
df = pd.DataFrame({'obj': things, 'obj_id': obj_ids})
t0 = time.time()
res = df[df['obj_id'].isin(some_ids)]['obj'].to_list()
t1 = time.time()
print('pandas df lookups:', t1-t0)

pandas df lookups: 2.985320568084717


In [8]:
# Maybe merge two dfs?
# nope, that's worse.
left_df = pd.DataFrame({'obj_id': some_ids})
t0 = time.time()
left_df.merge(df, on='obj_id', how='left')['obj'].to_list()
t1 = time.time()
print(t1-t0)

8.117634057998657


In [44]:
# OK, let's try REALLY basic numpy. If this doesn't work nothing will.
# Nice! Looks like a ~4x speedup, as long as we can use this mask efficiently. 

def in_it_time(main, subset):
    main_arr = np.array(main)
    subset_arr = np.array(subset)
    t0 = time.time()
    np.in1d(a2, a1, assume_unique=True)
    t1 = time.time()
    return t1-t0

print('neither sorted', in_it_time(obj_ids, some_ids))
print('little sorted', in_it_time(obj_ids, sorted(some_ids)))
print('big sorted', in_it_time(sorted(obj_ids), some_ids))
print('both sorted', in_it_time(sorted(obj_ids), sorted(some_ids)))


neither sorted 0.6735236644744873
little sorted 0.5764756202697754
big sorted 0.47378015518188477
both sorted 0.450084924697876


In [55]:
# what about this "sortednp" library, is it good?
sort_all = np.array(sorted(obj_ids))
sort_sub = np.array(sorted(some_ids))

t0 = time.time()
m = snp.intersect(sort_sub, sort_all, indices=True)
t1 = time.time()
print(t1-t0)
# NOW THAT'S MORE LIKE IT
# 0.09s let's gooooooo

0.09005618095397949


In [75]:
t0 = time.time()
df.iloc[m[1][1]]['obj']
t1 = time.time()
print('df lookup on that', t1-t0)
# WHAT NOW BITCHES


df lookup on that 0.3775930404663086


In [77]:
# all together now
n_runs = 10
t_tot = 0
for _ in range(n_runs):
    t0 = time.time()
    m = snp.intersect(sort_sub, sort_all, indices=True)
    winna = df.iloc[m[1][1]]['obj']
    t1 = time.time()
    t_tot += (t1-t0)/n_runs
print(t_tot)

print(len(winna))

0.39373033046722417
9932565


In [82]:
print('🎉', round(2.361854/0.3937, 2), 'x speedup 🎉')

🎉 6.0 x speedup 🎉


In [ ]:
# here are some other junks I tried that were bad.

In [40]:
# Maybe use the mask on that df?
t0 = time.time()
mask = np.in1d(a2, a1, assume_unique=True)
ls = df[mask]['obj'].to_list()
t1 = time.time()
print(t1-t0)

1.1768853664398193


In [39]:
# ok, looks like you can actually have a numpy array of objects directly. That 
# any better than a df?
# it is a little actually
t0 = time.time()
obj_arr = np.array(things)
t1 = time.time()
print('making the obj array took', t1-t0)

t0 = time.time()
mask = np.in1d(a2, a1, assume_unique=True)
wat = obj_arr[mask]
t1 = time.time()
print('', t1-t0)

# so, 10M per second
# that's around 0.1ms per item
# 0.5ms to find the mask, then the other 0.5ms to get the items

making the obj array took 9.505001068115234
0.9139652252197266


In [32]:
# desperation time
# what if we did like... a list comprehension
# nah it worse
t0 = time.time()
mask = np.in1d(a2, a1, assume_unique=True)
ok = [things[i] for i in range(len(things)) if mask[i]]
t1 = time.time()
print(t1-t0)


1.8870890140533447


In [27]:
obj_arr[0]

In [29]:
print(int('7fbe42dfa400', 16))
print(id(things[0]))

140455142466560
140455142466560


In [17]:
# it's not a huge speedup... about the same as using a list. Damn.
2.36 / 1.1028

2.140007254261879